# General Imports

In [2]:
from sklearn.datasets import make_regression
import numpy as np
import warnings

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_predict
from sklearn.inspection import permutation_importance

import subprocess
import sys

import plotly.graph_objects as go
from bayes_opt import BayesianOptimization
import matplotlib.pyplot as plt


Disable warnings

In [3]:
# Disable all warnings
warnings.filterwarnings("ignore")

# Enable warnings again
## warnings.filterwarnings("default")

In [4]:
# Generate regression toy data
n_samples = 1000
n_features = 5
X, y = make_regression(n_samples=n_samples, n_features=n_features, random_state=42)

# Split data into training and testing sets
train_ratio = 0.8
train_size = int(train_ratio * n_samples)

X_train = X[:train_size]
y_train = y[:train_size]

# Test data
X_test = X[train_size:]
y_test = y[train_size:]

In [5]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Assuming you have X_train, y_train defined for regression
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return -scores.mean()

In [6]:

#results = dt_bo.maximize(n_iter=5, init_points=20)

In [7]:
all_models = {}

# MLR

**feature selection**

In [8]:
# Assuming you have X_train and y_train defined for training data

# Initialize the Linear Regression model
lr_model = LinearRegression()

# Fit the Linear Regression model to the training data
lr_model.fit(X_train, y_train)

# Get feature coefficients
coefficients = lr_model.coef_

# Create a list of feature names or indices paired with their coefficients
feature_coefficients = list(zip(range(X_train.shape[1]), coefficients))

# Sort the features based on absolute coefficient values in descending order
feature_coefficients.sort(key=lambda x: abs(x[1]), reverse=True)

# Print the ranked feature coefficients
print("Feature Coefficients:")
for feature_index, coefficient in feature_coefficients:
    print(f"Feature {feature_index}: {coefficient}")

Feature Coefficients:
Feature 2: 46.07121713482753
Feature 3: 28.6279862111941
Feature 4: 24.74629812331462
Feature 1: 18.993474366101992
Feature 0: 16.823657910849178


**Hyperparameter optimalisatie**
#heb deze even uitgecomment omdat hij breekt als er geen min en max values zijn gedefinieerd

In [9]:
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install("bayesian-optimization")

# Define the MLR model evaluation function using cross-validation
def evaluate_mlr_model(fit_intercept):
    # Convert fit_intercept to a boolean value
    fit_intercept = bool(fit_intercept)
    
    # Create and configure the MLR model
    model = LinearRegression(fit_intercept=fit_intercept)
    
    # Perform cross-validation on the training data
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    
    # Return the negative mean squared error (Bayesian Optimization maximizes the objective)
    return np.mean(scores)

# Define the parameter ranges for Bayesian Optimization
params_ranges = {
    'fit_intercept': (0, 1)
}

# Perform Bayesian Optimization
mlr_bo = BayesianOptimization(f=evaluate_mlr_model, pbounds=params_ranges)
mlr_bo.maximize(n_iter=10, init_points=5)

# Get the best hyperparameters
best_params = mlr_bo.max['params']
best_fit_intercept = bool(best_params['fit_intercept'])

# Create the best MLR model with the tuned hyperparameters
best_model_mlr = LinearRegression(fit_intercept=best_fit_intercept)

# Fit the best model to the training data
best_model_mlr.fit(X_train, y_train)

# Calculate evaluation metrics
y_pred = best_model_mlr.predict(X_train)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
mse = mean_squared_error(y_train, y_pred)
mae = mean_absolute_error(y_train, y_pred)
aed = np.abs(y_train.mean() - y_pred)
r2 = r2_score(y_train, y_pred)
n = len(X_train)
k = X_train.shape[1]
r2_adj = 1 - (1 - r2) * ((n - 1) / (n - k - 1))

# Print the evaluation metrics
print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print("AED:", aed)
print("R2:", r2)
print("Adjusted R2:", r2_adj)

|   iter    |  target   | fit_in... |
-------------------------------------
| 1         | -2.967e-2 | 0.6536    |
| 2         | -2.967e-2 | 0.2347    |
| 3         | -2.967e-2 | 0.5823    |
| 4         | -2.967e-2 | 0.7304    |
| 5         | -2.967e-2 | 0.972     |
| 6         | -2.967e-2 | 0.0001533 |
| 7         | -2.967e-2 | 0.000163  |
| 8         | -2.967e-2 | 0.7045    |
| 9         | -2.967e-2 | 1.0       |
| 10        | -2.967e-2 | 6.419e-05 |
| 11        | -2.967e-2 | 0.9998    |
| 12        | -2.967e-2 | 0.3462    |
| 13        | -2.967e-2 | 0.4208    |
| 14        | -2.967e-2 | 0.07841   |
| 15        | -2.967e-2 | 0.4071    |
RMSE: 3.450179238121139e-14
MSE: 1.1903736775162163e-27
MAE: 2.6896141563925723e-14
AED: [2.69338913e+01 5.99685223e+01 8.07433456e+01 5.92404230e+01
 2.22614492e+00 1.35040308e+02 5.52997213e+01 5.20146668e+01
 2.43138526e+01 3.18940377e+01 1.10641570e+01 6.96546712e+01
 1.37617496e+01 6.04395798e+01 1.20692441e+02 9.78251954e+00
 6.57532103e+01 1.495

# Support Vector Machines

**Feature selection**

In [10]:
# Assuming you have X_train and y_train defined for training data

# Initialize the SVM model
svm_model = SVR(kernel='rbf')  # Replace 'rbf' with your desired kernel

# Fit the SVM model to the training data
svm_model.fit(X_train, y_train)

# Compute permutation importances
result = permutation_importance(svm_model, X_train, y_train, n_repeats=10, random_state=42)

# Get feature importances
importances = result.importances_mean

# Create a list of feature names or indices paired with their importances
feature_importances = list(zip(range(X_train.shape[1]), importances))

# Sort the features based on importance in descending order
feature_importances.sort(key=lambda x: x[1], reverse=True)

# Print the ranked feature importances
print("Feature Importances:")
for feature_index, importance in feature_importances:
    print(f"Feature {feature_index}: {importance}")

Feature Importances:
Feature 2: 0.5100907245192264
Feature 3: 0.1812236125303029
Feature 4: 0.13843292474891247
Feature 1: 0.10140750241353605
Feature 0: 0.07508208211413692


**Hyperparameter optimalisatie**

In [11]:
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install("bayesian-optimization")
 
# Define the SVM model evaluation function using cross-validation
def evaluate_svm_model(C, epsilon, gamma):
    # Create and configure the SVM model
    model = SVR(C=C, epsilon=epsilon, gamma=gamma)
    
    # Perform cross-validation on the training data
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    
    # Return the negative mean squared error (Bayesian Optimization maximizes the objective)
    return np.mean(scores)

# Define the parameter ranges for Bayesian Optimization
params_ranges = {
    'C': (0.1, 10),
    'epsilon': (0.01, 1),
    'gamma': (0.001, 0.1)
}

# Perform Bayesian Optimization
svm_bo = BayesianOptimization(f=evaluate_svm_model, pbounds=params_ranges)
svm_bo.maximize(n_iter=10, init_points=5)

# Get the best hyperparameters
best_params = svm_bo.max['params']
best_C = best_params['C']
best_epsilon = best_params['epsilon']
best_gamma = best_params['gamma']

# Create the best SVM model with the tuned hyperparameters
best_model_svm = SVR(C=best_C, epsilon=best_epsilon, gamma=best_gamma)

# Fit the best model to the training data
best_model_svm.fit(X_train, y_train)

# Calculate evaluation metrics
y_pred = best_model_svm.predict(X_train)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
mse = mean_squared_error(y_train, y_pred)
mae = mean_absolute_error(y_train, y_pred)
aed = np.abs(y_train.mean() - y_pred)
r2 = r2_score(y_train, y_pred)
n = len(X_train)
k = X_train.shape[1]
r2_adj = 1 - (1 - r2) * ((n - 1) / (n - k - 1))

# Print the evaluation metrics
print("RMSE:", rmse)
print("MSE:", mse)
print("MAE:", mae)
print("AED:", aed)
print("R2:", r2)
print("Adjusted R2:", r2_adj)

|   iter    |  target   |     C     |  epsilon  |   gamma   |
-------------------------------------------------------------
| 1         | -133.4    | 5.473     | 0.1113    | 0.04638   |
| 2         | -318.8    | 5.011     | 0.9129    | 0.09898   |
| 3         | -3.115e+0 | 0.3718    | 0.2997    | 0.03761   |
| 4         | -238.2    | 6.436     | 0.524     | 0.08268   |
| 5         | -134.2    | 3.685     | 0.9182    | 0.04048   |
| 6         | -3.004e+0 | 10.0      | 0.01      | 0.001     |
| 7         | -369.8    | 3.697     | 0.01      | 0.1       |
| 8         | -263.4    | 7.288     | 0.01      | 0.1       |
| 9         | -3.27e+03 | 7.618     | 1.0       | 0.001     |
| 10        | -450.2    | 2.465     | 1.0       | 0.1       |
| 11        | -3.87e+03 | 2.544     | 0.01      | 0.001     |
| 12        | -289.0    | 3.647     | 0.895     | 0.07841   |
| 13        | -3.65e+03 | 4.354     | 0.4484    | 0.001     |
| 14        | -3.06e+03 | 0.2839    | 0.1493    | 0.08911   |
| 15    

# Random Forest

**Feature selection**

In [12]:
# Assuming you have X_train and y_train defined for training data

# Initialize the Random Forest model
rf_model = RandomForestRegressor()

# Fit the Random Forest model to the training data
rf_model.fit(X_train, y_train)

# Get feature importances
importances = rf_model.feature_importances_

# Create a list of feature names or indices paired with their importances
feature_importances = list(zip(range(X_train.shape[1]), importances))

# Sort the features based on importance in descending order
feature_importances.sort(key=lambda x: x[1], reverse=True)

# Print the ranked feature importances
print("Feature Importances:")
for feature_index, importance in feature_importances:
    print(f"Feature {feature_index}: {importance}")


Feature Importances:
Feature 2: 0.5476675235912296
Feature 3: 0.17758579482865414
Feature 4: 0.13660830867702592
Feature 1: 0.07981194006155314
Feature 0: 0.05832643284153729


**Hyperparameter optimalisatie**

In [13]:
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install("bayesian-optimization")

params_ranges = {
    'n_estimators': (10, 100),
    'max_depth': (1, 20),
    'min_samples_leaf': (1, 10),
    'min_weight_fraction_leaf': (0.0, 0.5),
    'max_features': (0.1, 1),
    'max_leaf_nodes': (10, 100)
}

# Example usage with Random Forest
model = RandomForestRegressor
dt_bo = BayesianOptimization(f=lambda n_estimators, max_depth, min_samples_leaf, min_weight_fraction_leaf,
                                    max_features, max_leaf_nodes: bo_params_generic(model, {
                                        'n_estimators': int(round(n_estimators)),
                                        'max_depth': int(round(max_depth)),
                                        'min_samples_leaf': round(min_samples_leaf),
                                        'min_weight_fraction_leaf': min_weight_fraction_leaf,
                                        'max_features': max_features,
                                        'max_leaf_nodes': int(round(max_leaf_nodes))
                                    }, X_train, y_train),
                             pbounds=params_ranges)
results = dt_bo.maximize(n_iter=5, init_points=20)
params = dt_bo.max['params']

# Creating a model with the best hyperparameters
best_model_random_forest = model(
    n_estimators=int(round(params['n_estimators'])),
    max_depth=int(round(params['max_depth'])),
    min_samples_leaf=round(params['min_samples_leaf']),
    min_weight_fraction_leaf=params['min_weight_fraction_leaf'],
    max_features=params['max_features'],
    max_leaf_nodes=int(round(params['max_leaf_nodes']))
)

# Fit the model
best_model_random_forest.fit(X_train, y_train)

# Calculate evaluation metrics
mse_scores = -cross_val_score(best_model_random_forest, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
mae_scores = -cross_val_score(best_model_random_forest, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
aed_scores = np.abs(y_train.mean() - best_model_random_forest.predict(X_train))
r2_scores = cross_val_score(best_model_random_forest, X_train, y_train, cv=5, scoring='r2')
n = len(X_train)
k = X_train.shape[1]
r2_adj_scores = 1 - (1 - r2_scores) * ((n - 1) / (n - k - 1))
rmse_scores = np.sqrt(-mse_scores)

# Print the evaluation metrics
print("Average MSE:", np.mean(mse_scores))
print("Average MAE:", np.mean(mae_scores))
print("Average AED:", np.mean(aed_scores))
print("Average R2:", np.mean(r2_scores))
print("Average Adjusted R2:", np.mean(r2_adj_scores))
print("Average RMSE:", np.mean(rmse_scores))

|   iter    |  target   | max_depth | max_fe... | max_le... | min_sa... | min_we... | n_esti... |
-------------------------------------------------------------------------------------------------
| 1         | 54.52     | 4.631     | 0.2522    | 29.18     | 6.777     | 0.3139    | 69.78     |
| 2         | 37.24     | 19.59     | 0.7382    | 44.0      | 5.516     | 0.1045    | 67.43     |
| 3         | 56.01     | 3.066     | 0.2401    | 10.44     | 6.246     | 0.4511    | 41.41     |
| 4         | 51.98     | 8.69      | 0.5017    | 86.61     | 5.844     | 0.3199    | 11.42     |
| 5         | 50.36     | 4.605     | 0.5704    | 53.13     | 4.588     | 0.3065    | 99.17     |
| 6         | 53.18     | 2.212     | 0.5119    | 81.13     | 5.34      | 0.3522    | 93.28     |
| 7         | 53.34     | 6.494     | 0.465     | 53.25     | 6.784     | 0.4883    | 88.41     |
| 8         | 37.5      | 13.77     | 0.1875    | 51.16     | 7.42      | 0.05155   | 56.86     |
| 9         | 50.42 

# neural network

**Feature selection**

In [14]:
# Assuming you have X and y defined for the dataset

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the neural network regressor
nn_model = MLPRegressor(hidden_layer_sizes=(10, 10))  # Adjust the architecture as needed

# Fit the neural network model to the training data
nn_model.fit(X_train_scaled, y_train)

# Perform feature selection using Recursive Feature Elimination (RFE)
selector = RFE(estimator=nn_model, n_features_to_select=10)  # Adjust n_features_to_select as needed
selector.fit(X_train_scaled, y_train)

# Transform the training and testing sets to keep only the selected features
X_train_selected = selector.transform(X_train_scaled)
X_test_selected = selector.transform(X_test_scaled)

# Print the selected feature support
selected_support = selector.support_
print("Selected Feature Support:")
print(selected_support)

Selected Feature Support:
[ True  True  True  True  True]


**Hyperparameter optimalisatie**

In [15]:
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install("bayesian-optimization")

params_ranges = {
    'hidden_layer_sizes': (10, 100),
    'alpha': (0.0001, 0.1),
    'learning_rate_init': (0.001, 0.1),
    'max_iter': (100, 1000),
}

# Example usage with Neural Network
model = MLPRegressor
dt_bo = BayesianOptimization(f=lambda hidden_layer_sizes, alpha, learning_rate_init, max_iter:
                                    bo_params_generic(model, {
                                        'hidden_layer_sizes': (int(round(hidden_layer_sizes)),),
                                        'alpha': alpha,
                                        'learning_rate_init': learning_rate_init,
                                        'max_iter': int(round(max_iter))
                                    }, X_train, y_train),
                             pbounds=params_ranges)

results = dt_bo.maximize(n_iter=5, init_points=20)
params = dt_bo.max['params']

# Creating a model with the best hyperparameters
best_model_neural_network = model(
    hidden_layer_sizes=(int(round(params['hidden_layer_sizes'])),),
    alpha=params['alpha'],
    learning_rate_init=params['learning_rate_init'],
    max_iter=int(round(params['max_iter']))
)

# Fit the model
best_model_neural_network.fit(X_train, y_train)

# Calculate RMSE using cross-validation
rmse_scores = np.sqrt(-cross_val_score(best_model_neural_network, X_train, y_train, cv=5, scoring='neg_mean_squared_error'))
mse_scores = -cross_val_score(best_model_neural_network, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
mae_scores = -cross_val_score(best_model_neural_network, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
aed_scores = np.abs(y_train.mean() - best_model_neural_network.predict(X_train))
r2_scores = cross_val_score(best_model_neural_network, X_train, y_train, cv=5, scoring='r2')
n = len(X_train)
k = X_train.shape[1]
r2_adj_scores = 1 - (1 - r2_scores) * ((n - 1) / (n - k - 1))

# Print the evaluation metrics
print("Average RMSE:", np.mean(rmse_scores))
print("Average MSE:", np.mean(mse_scores))
print("Average MAE:", np.mean(mae_scores))
print("Average AED:", np.mean(aed_scores))
print("Average R2:", np.mean(r2_scores))
print("Average Adjusted R2:", np.mean(r2_adj_scores))

|   iter    |  target   |   alpha   | hidden... | learni... | max_iter  |
-------------------------------------------------------------------------
| 1         | 0.1759    | 0.01123   | 11.19     | 0.06432   | 848.6     |
| 2         | 0.2815    | 0.03808   | 50.57     | 0.01991   | 656.6     |
| 3         | 0.326     | 0.03691   | 87.75     | 0.01114   | 473.7     |
| 4         | 0.1501    | 0.07709   | 81.32     | 0.07718   | 683.6     |
| 5         | 0.1848    | 0.09488   | 78.17     | 0.07513   | 975.5     |
| 6         | 0.1623    | 0.04601   | 79.39     | 0.09305   | 552.9     |
| 7         | 0.2713    | 0.02331   | 48.97     | 0.03114   | 639.0     |
| 8         | 0.1725    | 0.01666   | 97.04     | 0.07032   | 236.4     |
| 9         | 0.2258    | 0.08563   | 94.34     | 0.03049   | 197.6     |
| 10        | 0.5415    | 0.05699   | 16.38     | 0.05694   | 156.5     |
| 11        | 0.1892    | 0.06427   | 34.23     | 0.09808   | 444.8     |
| 12        | 0.2921    | 0.09856   | 

In [16]:
# Append model to list
all_models['Random forest'] = best_model_random_forest
all_models['SVM'] = best_model_svm
all_models['Neural Network'] = best_model_neural_network
all_models['MLR']= best_model_mlr

In [17]:
# Evaluation metric labels
metric_labels = ['RMSE', 'MSE', 'MAE', 'AED', 'R2', 'Adjusted R2']

# Calculate evaluation metrics using cross-validation for each model
metrics = {
    'RMSE': lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)),
    'MSE': mean_squared_error,
    'MAE': mean_absolute_error,
    'AED': lambda y_true, y_pred: np.abs(np.mean(y_true) - y_pred),
    'R2': r2_score,
    'Adjusted R2': lambda y_true, y_pred: 1 - ((1 - r2_score(y_true, y_pred)) * (len(y_true) - 1) / (len(y_true) - X_train.shape[1] - 1))
}

model_names = ['Random Forest', 'SVM', 'Neural Network', 'MLR']

models = {
    'Random Forest': best_model_random_forest,
    'SVM': best_model_svm,
    'Neural Network': best_model_neural_network,
    'MLR': best_model_mlr
}

metric_scores = {metric: [] for metric in metric_labels}

for model_name in model_names:
    model = models[model_name]
    y_pred = cross_val_predict(model, X_train, y_train, cv=5)
    for metric in metric_labels:
        metric_scores[metric].append(metrics[metric](y_train, y_pred))

# Plotting the bar chart
fig = go.Figure()

for metric in metric_labels:
    fig.add_trace(go.Bar(
        x=model_names,
        y=metric_scores[metric],
        name=metric
    ))

# Updating the layout
fig.update_layout(
    title='Evaluation Metrics Comparison',
    xaxis_title='Models',
    yaxis_title='Scores',
    barmode='group'
)

# Display the plot
fig.show()
